In [ ]:
#base class
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    # computes the output Y of a layer for a given input X
    def forward_propagation(self, input):
        raise NotImplementedError

    # computes dE/dX for a given dE/dY (and update parameters if any)
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

In [ ]:

import numpy as np
weightff = []
# inherit from base class Layer
class FCLayer(Layer):
    # input_size = number of input neurons
    # output_size = number of output neurons
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    # returns output for a given input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    # computes dE/dW, dE/dB for a given output_error=dE/dY. Returns input_error=dE/dX.
    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        # dBias = output_error

        # update parameters
        self.weights -= learning_rate * weights_error

        self.bias -= learning_rate * output_error

        return input_error

In [ ]:
# inherit from base class Layer
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    # returns the activated input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    # Returns input_error=dE/dX for a given output_error=dE/dY.
    # learning_rate is not used because there is no "learnable" parameters.
    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

In [ ]:
# activation function and its derivative
def tanh(x):
    return np.tanh(x);

def tanh_prime(x):
    return 1-np.tanh(x)**2;

In [ ]:
# loss function and its derivative
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

In [ ]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    # predict output for given input
    def predict(self, input_data):
        # sample dimension first
        samples = len(input_data)
        result = []

        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    # train the network
    def fit(self, x_train, y_train, epochs, learning_rate):
        # sample dimension first
        samples = len(x_train)

        # training loop
        for i in range(epochs):
            err = 0
            w = []
            for j in range(samples):
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                # compute loss (for display purpose only)
                err += self.loss(y_train[j], output)

                # backward propagations
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):

                    error = layer.backward_propagation(error, learning_rate)

            # calculate average error on all samples
            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))


In [ ]:
from keras.datasets import mnist
from keras.utils import np_utils

# load MNIST from server
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# training data : 60000 samples
# reshape and normalize input data
x_train = x_train.reshape(x_train.shape[0], 1, 28*28)
x_train = x_train.astype('float32')
x_train /= 255
# encode output which is a number in range [0,9] into a vector of size 10
# e.g. number 3 will become [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
y_train = np_utils.to_categorical(y_train)

# same for test data : 10000 samples
x_test = x_test.reshape(x_test.shape[0], 1, 28*28)
x_test = x_test.astype('float32')
x_test /= 255
y_test = np_utils.to_categorical(y_test)

# Network
net = Network()
net.add(FCLayer(28*28, 100))                # input_shape=(1, 28*28)    ;   output_shape=(1, 100)
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(100, 50))                   # input_shape=(1, 100)      ;   output_shape=(1, 50)
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(50, 10))                   # input_shape=(1, 50)       ;   output_shape=(1, 10)
net.add(ActivationLayer(tanh, tanh_prime))

# train on 1000 samples
# as we didn't implemented mini-batch GD, training will be pretty slow if we update at each iteration on 60000 samples...
net.use(mse, mse_prime)

net.fit(x_train[0:1000], y_train[0:1000], epochs=35, learning_rate=0.1)

# test on 3 samples
testnum = 100
out = net.predict(x_test[0:testnum])

print("\n")
print("predicted values : ")
print(out, end="\n")
print("true values : ")
print(y_test[0:testnum])

11490434/11490434 [==============================] - 0s 0us/step
epoch 1/35   error=0.243210
epoch 2/35   error=0.102562
epoch 3/35   error=0.078610
epoch 4/35   error=0.065609
epoch 5/35   error=0.056104
epoch 6/35   error=0.048710
epoch 7/35   error=0.042586
epoch 8/35   error=0.037455
epoch 9/35   error=0.033283
epoch 10/35   error=0.029855
epoch 11/35   error=0.026957
epoch 12/35   error=0.024490
epoch 13/35   error=0.022492
epoch 14/35   error=0.020758
epoch 15/35   error=0.019286
epoch 16/35   error=0.017903
epoch 17/35   error=0.016788
epoch 18/35   error=0.015810
epoch 19/35   error=0.014988
epoch 20/35   error=0.014189
epoch 21/35   error=0.013534
epoch 22/35   error=0.012962
epoch 23/35   error=0.012516
epoch 24/35   error=0.012031
epoch 25/35   error=0.011634
epoch 26/35   error=0.011117
epoch 27/35   error=0.010623
epoch 28/35   error=0.010124
epoch 29/35   error=0.009746
epoch 30/35   error=0.009376
epoch 31/35   error=0.009062
epoch 32/35   error=0.008717
epoch 33/35   er

In [ ]:
pred = [0]*testnum
for i in range(testnum):
  for j in range(1):
    for k in range(10):
      if out[i][j][k] == max(out[i][j]):
        pred[i] = k+1
        print(k+1)

8
4
2
1
5
2
5
10
6
10
1
3
10
1
2
4
8
8
6
5
10
7
7
6
5
1
8
5
1
2
4
2
4
5
8
3
8
2
3
2
2
8
5
3
7
6
7
6
5
5
7
3
5
6
3
1
5
2
10
2
8
3
5
3
8
10
3
5
4
1
8
1
4
8
2
8
4
8
10
8
10
7
3
8
9
5
8
4
7
2
4
7
10
4
2
10
6
6
7
10


In [ ]:
np.arr

In [ ]:
for i in range(testnum):
  for j in range(10):
    if y_test[i][j] == 1:
      print(j+1)

8
3
2
1
5
2
5
10
6
10
1
7
10
1
2
6
10
8
4
5
10
7
7
6
5
1
8
5
1
2
4
2
4
5
8
3
8
2
3
2
2
8
5
3
4
6
2
3
5
5
7
4
6
6
7
1
5
2
10
6
8
9
10
4
8
5
7
5
4
1
8
1
3
10
2
8
4
3
10
8
8
7
3
8
9
5
8
4
7
2
4
7
10
4
2
5
2
8
7
10
